### Service Ticket Classification

* shows how to use structured output to automatically classify Service requests

In [1]:
from langchain_community.chat_models import ChatOCIGenAI
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
from enum import Enum
from typing import List

from config_private import COMPARTMENT_OCID

In [2]:
endpoint = "https://inference.generativeai.eu-frankfurt-1.oci.oraclecloud.com"

In [3]:
chat = ChatOCIGenAI(
    model_id="cohere.command-r-plus-08-2024",
    service_endpoint=endpoint,
    compartment_id=COMPARTMENT_OCID,
    provider="cohere",
    model_kwargs={
        "temperature": 0,
        "max_tokens": 600,
        "frequency_penalty": 1,
        "presence_penalty": 0,
        "top_p": 0.75,
        "top_k": 0,
        "seed": 1234,
    },
    auth_type="API_KEY",
)

In [4]:
class TicketCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    OTHER = "other"


class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"


class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

In [5]:
class TicketClassification(BaseModel):
    """
    This code defines a structured data model for classifying customer support tickets using Pydantic and Python's Enum class.
    It specifies categories, urgency levels, customer sentiments, and other relevant information as predefined options or constrained fields.
    This structure ensures data consistency, enables automatic validation, and facilitates easy integration with AI models and other parts of a support ticket system.
    """

    category: TicketCategory
    urgency: TicketUrgency
    sentiment: CustomerSentiment
    confidence: float = Field(
        ge=0, le=1, description="Confidence score for the classification"
    )
    key_information: List[str] = Field(
        description="List of key points extracted from the ticket"
    )
    suggested_action: str = Field(
        description="Brief suggestion for handling the ticket"
    )

In [6]:
SYSTEM_PROMPT = """
You are an AI assistant for a large e-commerce platform's customer support team.
Your role is to analyze incoming customer support tickets and provide structured information to help our team respond quickly and effectively.
Business Context:
- We handle thousands of tickets daily across various categories (orders, accounts, products, technical issues, billing).
- Quick and accurate classification is crucial for customer satisfaction and operational efficiency.
- We prioritize based on urgency and customer sentiment.
Your tasks:
1. Categorize the ticket into the most appropriate category (order_issue, account_access, product_inquiry, technical_support, billing, other).
2. Assess the urgency of the issue (low, medium, high, critical).
3. Determine the customer's sentiment (angry, frustrated, neutral, satisfied).
4. Extract key information that would be helpful for our support team.
5. Suggest an initial action for handling the ticket.
6. Provide a confidence score for your classification.
Remember:
- Be objective and base your analysis solely on the information provided in the ticket.
- If you're unsure about any aspect, reflect that in your confidence score.
- For 'key_information', extract specific details like order numbers, product names, or account issues.
- The 'suggested_action' should be a brief, actionable step for our support team.
Analyze the following customer support ticket and provide the requested information in the specified format.
"""

In [7]:
#
# this function is used to test
#
def classify_ticket(ticket_text: str) -> TicketClassification:
    structured_chat = chat.with_structured_output(schema=TicketClassification)
    return structured_chat.invoke(
        [
            SystemMessage(content=SYSTEM_PROMPT),
            HumanMessage(content=ticket_text),
        ]
    )

In [8]:
# Sample customer support tickets
ticket1 = """
I ordered a laptop from your store last week (Order #12345), but I received a tablet instead.
This is unacceptable! I need the laptop for work urgently. Please resolve this immediately or I'll have to dispute the charge.
"""

result1 = classify_ticket(ticket1)

print(result1.model_dump_json(indent=2))

{
  "category": "order_issue",
  "urgency": "high",
  "sentiment": "angry",
  "confidence": 1.0,
  "key_information": [
    "Order #12345",
    "laptop",
    "tablet"
  ],
  "suggested_action": "Check the order details and confirm the discrepancy. Contact the customer to apologise and arrange for the correct item to be delivered."
}


In [9]:
# Sample customer support tickets
ticket2 = """
Hello, I'm having trouble logging into my account. I've tried resetting my password, but I'm not receiving the reset email.
Can you please help me regain access to my account? I've been a loyal customer for years and have several pending orders.
"""

result2 = classify_ticket(ticket2)
print(result2.model_dump_json(indent=2))

{
  "category": "account_access",
  "urgency": "medium",
  "sentiment": "frustrated",
  "confidence": 1.0,
  "key_information": [
    "Customer is having trouble logging into their account",
    "They have tried resetting their password but are not receiving the reset email"
  ],
  "suggested_action": "Check the customer's email address on file and ensure it is correct. If it is, advise the customer to check their spam folder for the reset email."
}


In [10]:
# Sample customer support tickets
ticket3 = """
Hello, from time to time I have trouble accessing my MacBook, but it is not clear what the problem is..
The problem is that I'm not able to log-in, Have to try many times to resubmit my pwd.
What can I do?
It is the 10th time that my MacBook has problem. It is totally unacceptable.
"""

result3 = classify_ticket(ticket3)
print(result3.model_dump_json(indent=2))

{
  "category": "technical_support",
  "urgency": "medium",
  "sentiment": "angry",
  "confidence": 0.9,
  "key_information": [
    "Unable to log in to MacBook",
    "Password issues",
    "Recurring problem"
  ],
  "suggested_action": "Check for software updates and verify password settings."
}
